In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from monai.networks.nets import UNet
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from tqdm import tqdm

import osfrom pathlib import Path

import numpy as npfrom PIL import Image

### paths


In [ ]:
# Paths for patient folder structure
data_dir = "data"  # Organized dataset folder
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)
best_model_path = os.path.join(save_dir, "best_segmentation_model.pth")

val_dir = Path(data_dir) / "val"

# Define paths for train and validation splitstrain_dir = Path(data_dir) / "train"

### loading data

In [ ]:
class PatientFolderDataset(Dataset):
    def __init__(self, split_dir, image_size=(256, 256)):
        self.split_dir = Path(split_dir)
        self.image_size = image_size
        self.patient_folders = []
        
        # Find all patient folders in the split directory
        if self.split_dir.exists():
            self.patient_folders = [f for f in self.split_dir.iterdir() if f.is_dir()]
        
        # Validate each patient folder has exactly 2 images
        self.valid_patients = []
        for patient_folder in self.patient_folders:
            image_files = list(patient_folder.glob("*.jpg")) + list(patient_folder.glob("*.jpeg")) + list(patient_folder.glob("*.png"))
            

            if len(image_files) == 2:    val_dataset = None

                # Sort by file size - larger is usually original, smaller is mask    print(f"❌ Validation directory not found: {val_dir.absolute()}")

                img1, img2 = sorted(image_files, key=lambda x: x.stat().st_size, reverse=True)else:

                original_img = img1    val_dataset = PatientFolderDataset(val_dir)

                mask_img = img2if val_dir.exists():

                

                self.valid_patients.append((original_img, mask_img))    train_dataset = None

            print(f"❌ Training directory not found: {train_dir.absolute()}")

        print(f"Found {len(self.valid_patients)} valid patient cases in {split_dir}")else:

            train_dataset = PatientFolderDataset(train_dir)

    def __len__(self):if train_dir.exists():

        return len(self.valid_patients)# Create datasets

    

    def __getitem__(self, idx):        return {"image": image, "mask": mask}

        original_path, mask_path = self.valid_patients[idx]        

                mask = torch.tensor(mask).unsqueeze(0)    # Shape: (1, H, W)

        # Load and preprocess original image        image = torch.tensor(image).unsqueeze(0)  # Shape: (1, H, W)

        image = Image.open(original_path).convert('L')  # Convert to grayscale        # Convert to tensors and add channel dimension

        image = image.resize(self.image_size, Image.Resampling.LANCZOS)        

        image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]        mask = (mask > 0.5).astype(np.float32)  # Binarize

                mask = np.array(mask, dtype=np.float32) / 255.0  # Normalize to [0, 1]

        # Load and preprocess mask        mask = mask.resize(self.image_size, Image.Resampling.NEAREST)  # Use nearest for masks
        mask = Image.open(mask_path).convert('L')  # Convert to grayscale

### dataloaders init

In [ ]:
# Create data loaders
if train_dataset and len(train_dataset) > 0:
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    print(f"✅ Training loader created with {len(train_dataset)} samples")
else:

    print("❌ No training data found! Please run extract_malignant_cases.py first.")    val_loader = None

    train_loader = None    print("❌ No validation data found! Please run extract_malignant_cases.py first.")

else:

if val_dataset and len(val_dataset) > 0:    print(f"✅ Validation loader created with {len(val_dataset)} samples")
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

### model init

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
dice_metric = DiceMetric(include_background=False, reduction="mean")

### train loop

In [ ]:
# Check if we have data before starting training
if train_loader is None or val_loader is None:
    print("❌ Cannot start training without data!")
    print("\n📋 Next steps:")
    print("1. Run extract_malignant_cases.py to organize malignant patient folders")
    print("2. Check that the data directory structure is correct")
    print("3. Ensure patient folders contain exactly 2 images each (original + mask)")
else:
    print(f"🚀 Starting training with {len(train_dataset)} training samples and {len(val_dataset)} validation samples")
    
    num_epochs = 20
    best_dice = 0.0

    for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
    
    for batch in train_bar:
        images = batch["image"].to(device)
        masks = batch["mask"].to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        train_bar.set_postfix(loss=loss.item())

    # ===== Validation =====
    model.eval()
    dice_metric.reset()
    with torch.no_grad():
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]", leave=False)
        for batch in val_bar:
            images = batch["image"].to(device)
            masks = batch["mask"].to(device)
            
            outputs = model(images)
            preds = torch.sigmoid(outputs) > 0.5
            dice_metric(y_pred=preds, y=masks)
        
        val_dice = dice_metric.aggregate().item()
        dice_metric.reset()


    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {epoch_loss/len(train_loader):.4f} | Val Dice: {val_dice:.4f}")print(f"Best Dice score: {best_dice:.4f} | Saved at {best_model_path}")

print("Training complete!")

    # Save best model

    if val_dice > best_dice:        print(f"✅ Best model saved (Dice: {best_dice:.4f})")

        best_dice = val_dice        torch.save(model.state_dict(), best_model_path)